Let's do a little example similar to the Cannon problem.
Instead of spectra we can use time-series from finance.
We'll set up a small problem where we want to predict the values
of a stock (like the "flux") using the labels from other
two other stocks.
(Here we are working in time rather than wavelength.)
(Note: this is not a real predictive model for the stock market...do not use this!)

In [ ]:
#!pip install -U yfinance==0.2.44 

In [ ]:
import matplotlib.pylab as plt
import numpy as np
from scipy.optimize import minimize

import yfinance as yf
%matplotlib inline

Grab some data. Let's use Nike and Chipolte values as the labels and we'll see if we can come up with a model to predict Target.

In [ ]:
labels = ["NKE", "CMG"]
target = "TGT"
df = yf.download(labels, start="2024-01-01", end="2024-10-20")
for label in labels:
    (df[("Open", label)]/df[("Open", label)].median()).plot(alpha=0.8,label=label)
df_target = yf.download(target, start="2024-01-01", end="2024-10-20")
(df_target[("Open")]/df_target[("Open")].median() + 0.5).plot(linewidth=5,alpha=0.8,label=target)
plt.legend()

Let predict the Target using the price of the other stocks. We'll assume 
4 different timeseries (like 4 different "stars") here: open, close, high, low.

${\rm Target}_{\rm open}(t) = \theta_{0, t} +
    + \theta_{1, t} {\rm Nike}_{\rm open}(t) +  \theta_{2, t}{\rm Chipolte}_{\rm open}(t) \ldots $

${\rm Target}_{\rm close}(t) = \theta_{0, t} + 
    + \theta_{1, t} {\rm Nike}_{\rm close}(t) +  \theta_{2, t}{\rm Chipolte}_{\rm close}(t) \ldots $


${\rm Target}_{\rm high}(t) = \theta_{0, t} + \ldots $

$\vec y_t = {\mathbf{X}_t} \vec \theta_t$

with 

$${\mathbf{X}_t} = \begin{pmatrix}
1 &  {\rm Nike}_{\rm open}(t) & 
    {\rm Chipolte}_{\rm open} & \ldots \\
1 & {\rm Nike}_{\rm close}(t) & 
    {\rm Chipolte}_{\rm close} & \ldots\\
1 &  {\rm Nike}_{\rm high}(t) & 
    {\rm Chipolte}_{\rm high}& \ldots \\
1 &  {\rm Nike}_{\rm low}(t) & 
    {\rm Chipolte}_{\rm low} & \ldots
\end{pmatrix}
$$

and

$$\theta_t = \begin{pmatrix}
\theta_{0, t}\\
\theta_{1, t}\\
\theta_{2, t}
\end{pmatrix}
$$


Let's look at the data

In [ ]:
label_df = df.iloc[:][["Open", "Close", "High", "Low"]]
target_df = df_target.iloc[:][["Open", "Close", "High", "Low"]]
label_df

Now let's pick a time to predict. Again we're trying to predict 4 different values of Target stock using the labels from others stocks.

In [ ]:
t = 0
label_time_step_t = label_df.iloc[t].values.reshape(4, len(labels))
target_time_step_t = target_df.iloc[t].values.reshape(4, 1)

In [ ]:
label_time_step_t

In [ ]:
target_time_step_t

Set up the ${\mathbf{X}}$ matrix at this timestep:

In [ ]:
X_t = np.hstack([np.ones_like(target_time_step_t), label_time_step_t])
X_t

In [ ]:
θ_t, resid, _, _ = np.linalg.lstsq(X_t, target_time_step_t, rcond=-1)

In [ ]:
θ_t

Now let's predict four values of Target stock on this date:

In [ ]:
X_t @ θ_t 

In [ ]:
X_t @ θ_t  - target_time_step_t

In [ ]:
len_t = label_df.shape[0]
model_coeffs = np.zeros( ( len_t, len(labels)+1))

In [ ]:
for t in range(len_t):
    label_time_step_t = label_df.iloc[t].values.reshape(4, len(labels))
    target_time_step_t = target_df.iloc[t].values.reshape(4, 1)
    X_i = np.hstack([np.ones_like(target_time_step_t), label_time_step_t])
    coeffs, resid, _, _ = np.linalg.lstsq(X_i, target_time_step_t, rcond=-1)
    model_coeffs[t,:] = coeffs.T

n_model_parameters = model_coeffs.shape[1]

In [ ]:
plt.plot(label_df.index[0:50], model_coeffs[0:50, 1],label="CMG θ_1" )
plt.plot(label_df.index[0:50], model_coeffs[0:50, 2], label="NKE θ_2")
plt.hlines([0], xmin=label_df.index[0], xmax=label_df.index[50] )
plt.ylim(-25,25)
plt.legend()

Now let's try doing the fitting at one timestep by maximizing the likelihood

In [ ]:
def generate_target_at_one_timestep(θ, label_data):
    # calulate the target value given the labels
    X_t = np.hstack([1, label_data])
    return X_t @ θ

    
def neg_likelihood(θ_with_sigma, observed, label_data):

    # generate the (negative) log likelihood, given the labels and the observed data
    # and the model parameters  
    θ = θ_with_sigma[0:-1]
    sigma_2 = θ_with_sigma[-1]
    
    model_value = generate_target_at_one_timestep(θ, label_data)
    
    term1 = (-1/2)*(model_value - observed)**2/sigma_2
    term2 = np.log(sigma_2)
    return -1*(term1 + term2)

In [ ]:
# 
t0 = 0
cmg_ = 45.569000;
nke_ = 107.610001
tgt_ = 143.3288752
θ = model_coeffs[t0,:].reshape(1, n_model_parameters).T

label_data = np.array([cmg_, nke_]).reshape(2)
generate_target_at_one_timestep(θ, label_data)

sigma_2 = 0.01
θ_with_sigma = θ.reshape(3).tolist() + [sigma_2]

neg_likelihood(θ_with_sigma, tgt_, label_data)

In [ ]:
bnds = ((-1000, 1000), (-1000, 1000), (-1000, 1000), (0.0001, 1000000))
result = minimize(neg_likelihood, x0=(280, 10, -5, 0.1), 
                  args=(tgt_,label_data), bounds=bnds, method='SLSQP')

In [ ]:
result.x

In [ ]:
θ

In [114]:
%load_ext watermark

In [116]:
%watermark -v -p numpy,pandas,yfinance,scipy,matplotlib  -a "J Bloom" -d -t

Author: J Bloom

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.26.0

numpy     : 1.26.4
pandas    : 2.2.2
yfinance  : 0.2.44
scipy     : 1.13.1
matplotlib: 3.9.2

